# Does Bobby Always Win on "Beat Bobby Flay?"

Well, no, not exactly. Bobby is good. But he's not that good. Beating Bobby is still quite the accomplishment though. Let's dive into some fun insights from the show.

__Load Libraries__

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats
import random
import wordcloud

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%matplotlib inline

__Load Data File__

In [ ]:
bbf_df = pd.read_csv("Data/Beat_Bobby_Flay_Wikipedia_Data.csv", engine = "python")

In [ ]:
bbf_df.info()

## What dishes does Bobby most often win at?

In [ ]:
# subset to the contests Bobby won
winning_dishes = bbf_df.loc[bbf_df["BobbyDominates"] == 1]

# get the dishes column
winning_dishes = winning_dishes[["SecondRoundDish"]]

# get all dishes into a string of text
text = " ".join(dish for dish in winning_dishes.SecondRoundDish)

# Create and generate a word cloud image:
# wc = wordcloud.WordCloud().generate(text)
wc = wordcloud.WordCloud(max_font_size = 100, max_words = 100, background_color = "black").generate(text)

# Display the generated image:
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

## What dishes does Bobby most often lose at?

In [ ]:
# subset to the contests Bobby won
winning_dishes = bbf_df.loc[bbf_df["BobbyDominates"] == 0]

# get the dishes column
winning_dishes = winning_dishes[["SecondRoundDish"]]

# get all dishes into a string of text
text = " ".join(dish for dish in winning_dishes.SecondRoundDish)

# Create and generate a word cloud image:
wc = wordcloud.WordCloud(max_font_size = 100, max_words = 100, background_color = "black").generate(text)

# Display the generated image:
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

As "chicken" and "sandwich" seem to be dishes that are commonly made in the Second Round, it's tough to make a decisive opinion on what Bobby is really good or bad at cooking. A few things that stick out are that in the WordCloud of dishes Bobby has won, he is good at cooking beef and pork, and in the WordCloud of dishes Bobby has lost, he is not good at baking (i.e. cake).

## What is Bobby's overall record?

186 - 111. 

63% of the time, Bobby wins. Honestly this is lower than I thought, but still very impressive.

In [ ]:
# create indicator for episodes where Bobby has won
bbf_df["BobbyDominates"] = np.where(bbf_df["SecondRoundWinner"] == "Bobby Flay", 1, 0)

# group by the indicator
overall_record = bbf_df.groupby("BobbyDominates").agg({"BobbyDominates":["count"]})

# exclude first level of multi-indexed columns resulting from the groupby
overall_record.columns = overall_record.columns.droplevel(0)

# rename the column
overall_record.columns = ["Results"]

# show results
overall_record

0 = Bobby Loses  
1 = Bobby Dominates

In [ ]:
print("Bobby's winning percentage: ", 
      (overall_record.iloc[1,0] / (overall_record.iloc[0,0] + overall_record.iloc[1,0])).round(2)*100, "%")

## How has Bobby done over time?

Bobby has only had three losing seasons, and one .500 season.

Simply put, Bobby dominates. 

In [ ]:
# get results by season
seasonal_record = bbf_df.groupby(["Season"]).agg({"BobbyDominates":["mean"]}).round(4)

# reset the index
seasonal_record.reset_index(inplace = True)

# plot line graph
fig = plt.figure(figsize=(15,4))
ax1 = fig.add_subplot(111)

ax1.plot(seasonal_record["Season"], seasonal_record[("BobbyDominates", "mean")],
         color = "c",
         marker = "o")
ax1.axhline(y = 0.5, color = 'r', linestyle = '--')
plt.xticks(seasonal_record["Season"])
plt.xlabel("Seasons")
plt.ylabel("Winning Percentage")
plt.title("Bobby Wins Most Episodes Most Seasons")

## Wich Guest has appeared on the show the most?

Alex Guarnaschelli appears the most on the show.

In [ ]:
# split the Guests column into separate columns for every guest
guests_df = pd.concat([bbf_df["Guests"], bbf_df["Guests"].str.split(',', expand = True)], axis = 1)

# rename columns
guests_df.columns = ["Guests","Guest1","Guest2","Guest3"]

# stack guests 
guests_series = pd.concat([guests_df["Guest1"], 
                           guests_df["Guest2"], 
                           guests_df["Guest3"]],
                          axis = 0).dropna() # axis = 0 for rows

# strip whitespace so to not capture duplicates
guests_series = guests_series.str.strip()

# create dataframe
guests_df = pd.DataFrame({'Guests':guests_series})

# group by Guest and count
guests_df_grp = guests_df.groupby("Guests").agg({"Guests":["count"]})

# exclude first level of multi-indexed columns resulting from the groupby
guests_df_grp.columns = guests_df_grp.columns.droplevel(0)

# sort values
guests_df_grp = guests_df_grp.sort_values("count", ascending = False).reset_index()

# rename the column
guests_df_grp.columns = ["Guest", "Appearances"]

# specify top quantile (0 to 1)
quant = guests_df_grp.quantile(0.95)

# show which Guests have made the most appearances
guests_df_grp.loc[guests_df_grp["Appearances"] >= quant[0], :]

## What is the record of a Guest?

This has nothing to do with who beats Bobby or how to beat Bobby, but the Guests joke about beating Bobby because their choice of contestant beats Bobby.

Only 25 Guests have appeared five or more times on the show. Of these Guests, Valerie Bertinelli has the best winning percentage against Bobby. Valeria and Giada are the only ones with winning records. 

Looks like Bobby really wants Marc Murphy and Debi Mazar to be Guests on the show.

In [ ]:
# create indicator for Guests winning
bbf_df["GuestsDominate"] = np.where(bbf_df["SecondRoundWinner"] == "Bobby Flay", 0, 1)

# split the Guests column into separate columns for every guest
guests_record_df = pd.concat([bbf_df["Guests"], bbf_df["Guests"].str.split(',', expand = True)], axis = 1)

# rename columns
guests_record_df.columns = ["Guests","Guest1","Guest2","Guest3"]

# bind columns together
guests_record_df = pd.concat([guests_record_df, bbf_df["GuestsDominate"]], axis = 1)

# get Guest1 and GuestsDominate indicator
guest1_df = guests_record_df.loc[:, ["Guest1", "GuestsDominate"]]

# get Guest2 and GuestsDominate indicator
guest2_df = guests_record_df.loc[:, ["Guest2", "GuestsDominate"]]

# get Guest3 and GuestsDominate indicator
guest3_df = guests_record_df.loc[:, ["Guest3", "GuestsDominate"]]

# rename columns so they could be stacked
guest1_df.columns = ["Guest","GuestsDominate"]
guest2_df.columns = ["Guest","GuestsDominate"]
guest3_df.columns = ["Guest","GuestsDominate"]

# stack guests 
guests_record_df = pd.concat([guest1_df, 
                              guest2_df, 
                              guest3_df],
                             axis = 0).dropna() # axis = 0 for rows

# strip whitespace so to not capture duplicates
guests_record_df["Guest"] = guests_record_df["Guest"].str.strip()

# group by Guest and count
guests_record_grp = guests_record_df.groupby("Guest").agg({"Guest":["count"], "GuestsDominate":["mean"]})

# exclude first level of multi-indexed columns resulting from the groupby
guests_record_grp.columns = guests_record_grp.columns.droplevel(0)

# rename the column
guests_record_grp.columns = ["Appearances", "Guest's Winning %"]

# show the top winning percentages for Guests who have appeared at least five times
(guests_record_grp.loc[guests_record_grp["Appearances"] >= 5, :]
 .sort_values("Guest's Winning %", ascending = False)
 .reset_index())

## What is Bobby's record against other Food Network personalities?

Bobby has not beat a Guest/Food Network Personality.

That may sound shocking, however, contestants get to pick what he/she and Bobby get to cook in the second round. Realistically, Bobby doesn't have a chance at winning against other world-renowned chefs and his/her respective world-renowned dishes. 

In [ ]:
# split the Guests column into separate columns for every guest
guest_personalities_df = pd.concat([bbf_df["Guests"],
                                    bbf_df["Guests"].str.split(',', expand = True)],
                                   axis = 1) # axis = 1 for column bind

# rename columns
guest_personalities_df.columns = ["Guests","Guest1","Guest2","Guest3"]

# identify which episodes Bobby faces an Equal/Guest
guest_personalities_df["BobbyFacesPersonality"] = np.where(guest_personalities_df["Guest2"].isnull(), 1, 0)

# bind columns together
guest_personalities_df = pd.concat([guest_personalities_df, 
                                    bbf_df["BobbyDominates"],
                                    bbf_df["SecondRoundWinner"]],
                                   axis = 1)

# group by Food Network Personalities
guest_personalities_df.groupby("BobbyFacesPersonality").agg({"BobbyDominates":"mean"})

In [ ]:
guest_personalities_df.loc[guest_personalities_df["BobbyFacesPersonality"] == 1,
                           ["SecondRoundWinner","BobbyDominates"]]

Looks like these chefs have Bobby's number. 

## Who are the chefs that have beat Bobby multiple times?

Alex Guarnaschelli is the only chef to defeat Bobby TWICE!

In [ ]:
# get wins by contestant
mult_winner_grp = bbf_df.groupby("SecondRoundWinner").agg({"GuestsDominate":["sum"]})

# exclude first level of multi-indexed columns resulting from the groupby
mult_winner_grp.columns = mult_winner_grp.columns.droplevel(0)

# rename columns
mult_winner_grp.columns = ["Wins"]

# show the contestants who have at least two wins
(mult_winner_grp.loc[mult_winner_grp["Wins"] > 1, :]
 .sort_values("Wins", ascending = False)
 .reset_index())

## What Judge has appeared the most on the show?

Frank Prisinzano has appeared the most as a judge. 

In [ ]:
# split the Judges column into separate columns for every judge
judges_df = pd.concat([bbf_df["SecondRoundJudges"], 
                       bbf_df["SecondRoundJudges"].str.split(',', expand = True)], axis = 1)

# rename columns
judges_df.columns = ["Judges","Judge1","Judge2","Judge3"]

# stack judges
judges_series = pd.concat([judges_df["Judge1"], 
                           judges_df["Judge2"], 
                           judges_df["Judge3"]],
                          axis = 0).dropna() # axis = 0 for rows

# strip whitespace so to not capture duplicates
judges_series = judges_series.str.strip()

# create dataframe
judges_df = pd.DataFrame({'Judges': judges_series})

# group by Judges and count
judges_df_grp = judges_df.groupby("Judges").agg({"Judges":["count"]})

# exclude first level of multi-indexed columns resulting from the groupby
judges_df_grp.columns = judges_df_grp.columns.droplevel(0)

# sort values
judges_df_grp = judges_df_grp.sort_values("count", ascending = False).reset_index()

# rename columns
judges_df_grp.columns = ["Judge","Appearances"]

# specify top quantile (0 to 1)
quant = judges_df_grp.quantile(0.95)

# show which Guests have made the most appearances
judges_df_grp.loc[judges_df_grp["Appearances"] >= quant[0], :]

## Are there biased judges (for/against Bobby)?

When Tricia Williams appears as a judge, it seems the contestant wins  more often. For 11 appearances, that seems like some pretty biased results. Of course, correlation does not mean causation, and 11 appearances isn't enough to statistically prove bias. However, Bobby would probably prefer to have Tricia appear less often on the show. 

In [ ]:
# split the Guests column into separate columns for every guest
judges_record_df = pd.concat([bbf_df["SecondRoundJudges"], 
                              bbf_df["SecondRoundJudges"].str.split(',', expand = True)], axis = 1)

# rename columns
judges_record_df.columns = ["Judges","Judge1","Judge2","Judge3"]

# bind columns together
judges_record_df = pd.concat([judges_record_df, bbf_df["GuestsDominate"]], axis = 1)

# get Judge1 and GuestsDominate indicator
judge1_df = judges_record_df.loc[:, ["Judge1", "GuestsDominate"]]

# get Judge2 and GuestsDominate indicator
judge2_df = judges_record_df.loc[:, ["Judge2", "GuestsDominate"]]

# get Judge3 and GuestsDominate indicator
judge3_df = judges_record_df.loc[:, ["Judge3", "GuestsDominate"]]

# rename columns so they could be stacked
judge1_df.columns = ["Judge","GuestsDominate"]
judge2_df.columns = ["Judge","GuestsDominate"]
judge3_df.columns = ["Judge","GuestsDominate"]

# stack guests 
judges_record_df = pd.concat([judge1_df, 
                              judge2_df, 
                              judge3_df],
                             axis = 0).dropna() # axis = 0 for rows

# strip whitespace so to not capture duplicates
judges_record_df["Judge"] = judges_record_df["Judge"].str.strip()

# group by Judge and count
judges_record_grp = judges_record_df.groupby("Judge").agg({"Judge":["count"], "GuestsDominate":["mean"]})

# exclude first level of multi-indexed columns resulting from the groupby
judges_record_grp.columns = judges_record_grp.columns.droplevel(0)

# rename columns
judges_record_grp.columns = ["Appearances","Guest's Winning %"]

# show the top winning percentages for Judges who have appeared at least five times
(judges_record_grp.loc[judges_record_grp["Appearances"] >= 10, :]
 .sort_values("Guest's Winning %", ascending = False)
 .reset_index())

## How does Bobby do against contestants who have been judges?

Bobby is more human than god when he faces off against a contestant who is a judge on the show. Contestants who are judges win about half the time. 

In [ ]:
# split the Guests column into separate columns for every guest
contestant_record_df = pd.concat([bbf_df["FirstRoundContestants"], 
                                  bbf_df["FirstRoundContestants"].str.split(',', expand = True)], axis = 1)

# rename columns
contestant_record_df.columns = ["Contestant","Contestant1","Contestant2"]

# # strip whitespace so to not capture duplicates
# bbf_df["SecondRoundWinner"] = bbf_df["SecondRoundWinner"].str.strip()

# bind columns together
contestant_record_df = pd.concat([contestant_record_df, bbf_df["BobbyDominates"]], axis = 1)

# get Contestant1 and BobbyDominates indicator
contestant1_df = contestant_record_df.loc[:, ["Contestant1", "BobbyDominates"]]

# get Contestant2 and BobbyDominates indicator
contestant2_df = contestant_record_df.loc[:, ["Contestant2", "BobbyDominates"]]

# rename columns so they could be stacked
contestant1_df.columns = ["Contestant","BobbyDominates"]
contestant2_df.columns = ["Contestant","BobbyDominates"]

# stack contestants
contestant_record_df = pd.concat([contestant1_df, 
                                  contestant2_df],
                                 axis = 0).dropna() # axis = 0 for rows

# strip whitespace so to not capture duplicates
contestant_record_df["Contestant"] = contestant_record_df["Contestant"].str.strip()

# get a unique list of the judges
judges_uniq_df = judges_df.drop_duplicates()

# get contestants who have been judges (join)
contestants_merged_df = pd.merge(contestant_record_df, judges_uniq_df,
                                 left_on = "Contestant",
                                 right_on = "Judges",
                                 how = "left")

# get contestants who have been judges (indicator)
contestants_merged_df["IsJudge"] = np.where(contestants_merged_df["Contestant"] == 
                                            contestants_merged_df["Judges"], "Yes", "No")

# group by Judge and get winning percentage
contestants_merged_df = contestants_merged_df.groupby("IsJudge").agg({"BobbyDominates":["mean"]})

# exclude first level of multi-indexed columns resulting from the groupby
contestants_merged_df.columns = contestants_merged_df.columns.droplevel(0)

# rename columns
contestants_merged_df.columns = ["Contestant's Winning %"]

# show resulting table
contestants_merged_df